In [2]:
import pandas as pd

covid_data = pd.read_csv('data/covid_data.csv')
display(covid_data.head())

,date,province/state,country,confirmed,deaths,recovered
0,01/22/2020,Anhui,China,1.0,0.0,0.0
1,01/22/2020,Beijing,China,14.0,0.0,0.0
2,01/22/2020,Chongqing,China,6.0,0.0,0.0
3,01/22/2020,Fujian,China,1.0,0.0,0.0
4,01/22/2020,Gansu,China,0.0,0.0,0.0


In [4]:
vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]
display(vaccinations_data.head())

,country,date,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
0,Afghanistan,2021-02-22,0.0,0.0,0.0,NaN,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."


#### Возникает большое желание объединить таблицы. Для этого необходимо учитывать следующие нюансы: 
#### 1 В таблице covid_data необходимо предварительно рассчитать суммарное ежедневное число заболевших во всех провинциях/штатах в каждой стране.
#### 2 В таблицах не совпадает число стран, а иногда и их названия. 
#### При объединении таблиц по столбцу мы определённо теряем данные (в данной задаче потери незначительны). Избежать этого можно ручными преобразованиями данных — искать различия в названиях стран в таблицах и преобразовывать их. Однако это не является темой данного модуля.
#### 3 Таблицы имеют разные периоды наблюдений (вакцины появились позже, чем сам вирус). Объединив данные с типом inner, мы можем потерять большое количество наблюдений в таблице covid_data.
## ПРЕДОБРАБОТКА ДАННЫХ
#### Опираясь на замечания выше, выполним небольшую предобработку.
#### В таблице covid_data:

In [5]:
# Группируем таблицу по дате и названию страны и рассчитываем 
# суммарные показатели по всем регионам.
# Тем самым переходим от данных по регионам к данным по странам:
covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

In [6]:
# Преобразуем даты в формат datetime с помощью функции pd.to_datetime():
covid_data['date'] = pd.to_datetime(covid_data['date'])

In [7]:
# Создадим признак больных на данный момент (active). 
# Для этого вычтем из общего числа 
# зафиксированных случаев число смертей и число выздоровевших пациентов:
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

In [8]:
# Создадим признак ежедневного прироста числа заболевших, умерших и
# выздоровевших людей. Для этого отсортируем данные по названиям стран, 
# а затем по датам. После этого произведём группировку по странам и
# рассчитаем разницу между «вчера и сегодня» с помощью метода diff():
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

In [9]:
# Итоговый вид таблицы covid_data:
display(covid_data.head())

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered
11337,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,NaN,NaN,NaN
11570,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
11807,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12051,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12299,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [10]:
# В таблице vaccinations_data достаточно 
# будет преобразовать столбцы в формат datetime:
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

#### Итак, мы произвели некоторую предобработку данных. Дальнейшее объединение таблиц остаётся за вами (обязательно выполните задания ниже, прежде чем двигаться дальше).

In [11]:
# Задание 3.1 За какой период представлены данные в таблице covid_data? 
# В качестве ответа введите даты в формате datetime (без указания времени).
print(f"Данные о заболеваемости предоставлены от {covid_data['date'].min()} до  {covid_data['date'].max()}")

Данные о заболеваемости предоставлены от 2020-01-22 00:00:00 до  2021-05-29 00:00:00


In [12]:
# Задание 3.2 За какой период представлены данные в таблице vaccinations_data? В
# качестве ответа введите даты в формате datetime без указания времени.
print(f"Данные о вакцинации предоставлены от {vaccinations_data['date'].min()} до  {vaccinations_data['date'].max()}")

Данные о вакцинации предоставлены от 2020-12-02 00:00:00 до  2021-09-06 00:00:00


In [14]:
# Задание 3.3 С помощью метода merge() объедините таблицы 
# covid_data и vaccinations_data по столбцам date и country.
# Тип объединения выставьте так, чтобы в результирующую таблицу 
# попали только наблюдения за период, вычисленный в задании 3.1.
# То есть в результирующую таблицу должны попасть все записи из 
# таблицы covid_data 
# и из её пересечения с vaccinations_data, но не более. 
# Результат объединения занесите в переменную covid_df
covid_df = covid_data.merge(vaccinations_data, on=['date', 'country'], how='left')
# ! ! ! Сохраните таблицу covid_df — она понадобится нам в следующем юните.
# ! ! !
# ! ! !
print('Число строк: ', covid_df.shape[0])
print('Число столбцов: ', covid_df.shape[1])
print(covid_df.shape) # я сделал проще

Число строк:  86785
Число столбцов:  16
(86785, 16)


In [15]:
# Задание 3.4 Какова максимальная летальность в США 
# (United States) за весь период?
# Ответ округлите до второго знака после запятой.
covid_df['death_rate'] = covid_df['deaths'] / covid_df['confirmed'] * 100
covid_df['recover_rate'] = covid_df['recovered'] / covid_df['confirmed'] * 100
round(covid_df[covid_df['country'] == 'United States']['death_rate'].max(), 2)

7.19

In [16]:
# Задание 3.5 Чему равен средний процент выздоровевших в России (Russia)?
# Ответ округлите до второго знака после запятой.
round(covid_df[covid_df['country'] == 'Russia']['recover_rate'].mean(), 2)

67.06